# setup for colab

### enable GPU

You can enable the GPU in one of two ways: 

 * Select **`GPU`**  in the Edit->Notebook Settings for Hardware Accelerator   **OR**
 

 * Select **`GPU`**  in the Runtime->Change runtime type  for Hardware Accelerator 
 
 
 You should be able to now see the nvidia-smi diagnostic: 

In [ ]:
!nvidia-smi

Sun Nov 22 15:10:48 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

###  install bert-sklearn

In [ ]:
!git clone -b master https://github.com/charles9n/bert-sklearn
!cd bert-sklearn; pip install .
import os
os.chdir("bert-sklearn")
print(os.listdir())

Cloning into 'bert-sklearn'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 259 (delta 3), reused 3 (delta 0), pack-reused 247
Receiving objects: 100% (259/259), 519.36 KiB | 14.43 MiB/s, done.
Resolving deltas: 100% (125/125), done.
Processing /content/bert-sklearn
     |████████████████████████████████| 133kB 7.3MB/s 
     |████████████████████████████████| 6.9MB 12.3MB/s 
     |████████████████████████████████| 71kB 9.5MB/s 
  Created wheel for bert-sklearn: filename=bert_sklearn-0.3.1-cp36-none-any.whl size=54235 sha256=a5ddd0843d39b966f256974d614d1da331c2c22efe19386d818fcc90c55f2b6d
  Stored in directory: /root/.cache/pip/wheels/61/95/c6/5790aae8fb377f5ff356dbe58205aab28858595d6bff8197d0
Successfully built bert-sklearn
ERROR: botocore 1.19.23 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.
['LICENSE', 'setup.py

In [ ]:
import torch
print('pytorch version:', torch.__version__)
print('GPU:',torch.cuda.get_device_name(0))

pytorch version: 1.7.0+cu101
GPU: Tesla T4


In [ ]:
import os
import numpy as np
import pandas as pd
import re
from sklearn.metrics import classification_report
from tqdm import tqdm

from bert_sklearn import BertClassifier

In [ ]:
%%bash
if [[ ! -f ./zooniverse_all_data.csv ]]; then 
   wget https://raw.githubusercontent.com/aatishsuman/zooniverse-online-learning/master/data/zooniverse_all_data.csv
fi

if [[ ! -f ./zooniverse_train.csv ]]; then 
   wget https://raw.githubusercontent.com/aatishsuman/zooniverse-online-learning/master/data/zooniverse_train.csv
fi

if [[ ! -f ./sentiment_dataset.csv ]]; then 
   wget https://raw.githubusercontent.com/aatishsuman/zooniverse-online-learning/master/data/sentiment_dataset.csv
fi

--2020-11-22 15:11:01--  https://raw.githubusercontent.com/aatishsuman/zooniverse-online-learning/master/data/zooniverse_all_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17968115 (17M) [text/plain]
Saving to: ‘zooniverse_all_data.csv’

     0K .......... .......... .......... .......... ..........  0% 6.39M 3s
    50K .......... .......... .......... .......... ..........  0% 6.51M 3s
   100K .......... .......... .......... .......... ..........  0% 47.3M 2s
   150K .......... .......... .......... .......... ..........  1% 24.7M 2s
   200K .......... .......... .......... .......... ..........  1% 10.1M 2s
   250K .......... .......... .......... .......... ..........  1% 82.0M 1s
   300K .......... .......... .......... .......... ..........  1% 39

In [ ]:
all_data = pd.read_csv('zooniverse_all_data.csv')
test_df = pd.read_csv('zooniverse_train.csv')
train_df = pd.read_csv('sentiment_dataset.csv')

print(train_df.shape, test_df.shape, all_data.shape)
print(test_df.columns)
train_df.head()

(14640, 15) (400, 16) (90501, 8)
Index(['web-scraper-order', 'text', 'section', 'topic', 'urls', 'userMentions',
       'isReply', 'emojis', 'Expression of emotions', 'Use of humor',
       'Self-disclosure', 'Referring explicitly to others' messages',
       'Asking questions',
       'Complimenting, expressing appreciation or criticism',
       'Expressing agreement or disagreement', 'Phatics, salutations'],
      dtype='object')


,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [ ]:
def clean_tweets(text):
  user_pattern = '@[A-Za-z0-9_]+'
  http_pattern = 'https?://[^ ]+'
  www_pattern = 'www.[^ ]+'
  combined_pattern = '|'.join((user_pattern, http_pattern, www_pattern))
  return re.sub(combined_pattern, '', text).lower()

train_df['label'] = train_df['airline_sentiment'].apply(lambda label: 0 if label == 'neutral' else 1)
train_df = train_df[train_df['label'] == 0].append(train_df[train_df['label'] == 1].sample(sum(train_df['label'] == 0), axis=0)).reset_index(drop=True)
X_train, y_train = train_df['text'].apply(clean_tweets), train_df['airline_sentiment'].apply(lambda label: 0 if label == 'neutral' else 1)
print(len(X_train), len(y_train))
np.unique(y_train, return_counts=True)

6198 6198


(array([0, 1]), array([3099, 3099]))

In [ ]:
model = BertClassifier(max_seq_length = 128, epochs = 3, train_batch_size=32, validation_fraction=0.2)
print(model)
model.fit(X_train, y_train)

Building sklearn text classifier...
BertClassifier(bert_config_json=None, bert_model='bert-base-uncased',
               bert_vocab=None, do_lower_case=None, epochs=3, eval_batch_size=8,
               fp16=False, from_tf=False, gradient_accumulation_steps=1,
               ignore_label=None, label_list=None, learning_rate=2e-05,
               local_rank=-1, logfile='bert_sklearn.log', loss_scale=0,
               max_seq_length=128, num_mlp_hiddens=500, num_mlp_layers=0,
               random_state=42, restore_file=None, train_batch_size=32,
               use_cuda=True, validation_fraction=0.2, warmup_proportion=0.1)
Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint

train data size: 4959, validation data size: 1239





Epoch 1, Train loss: 0.5061, Val loss: 0.3679, Val accy: 83.86%





Epoch 2, Train loss: 0.2660, Val loss: 0.3870, Val accy: 83.45%





Epoch 3, Train loss: 0.1336, Val loss: 0.4590, Val accy: 83.54%



BertClassifier(bert_config_json=None, bert_model='bert-base-uncased',
               bert_vocab=None, do_lower_case=True, epochs=3, eval_batch_size=8,
               fp16=False, from_tf=False, gradient_accumulation_steps=1,
               ignore_label=None, label_list=array([0, 1]), learning_rate=2e-05,
               local_rank=-1, logfile='bert_sklearn.log', loss_scale=0,
               max_seq_length=128, num_mlp_hiddens=500, num_mlp_layers=0,
               random_state=42, restore_file=None, train_batch_size=32,
               use_cuda=True, validation_fraction=0.2, warmup_proportion=0.1)

In [ ]:
test_df['prediction'] = model.predict(test_df['text'].apply(clean_tweets))
test_df['prediction'] = (test_df['prediction'].astype(bool) | ~pd.isnull(test_df['emojis'])).astype(int)
print(classification_report(test_df['Expression of emotions'], test_df['prediction'], target_names=['0', '1']))


              precision    recall  f1-score   support

           0       0.95      0.84      0.89       350
           1       0.37      0.66      0.47        50

    accuracy                           0.82       400
   macro avg       0.66      0.75      0.68       400
weighted avg       0.87      0.82      0.84       400



In [ ]:
test_df

,web-scraper-order,text,section,topic,urls,userMentions,isReply,emojis,Expression of emotions,Use of humor,Self-disclosure,Referring explicitly to others' messages,Asking questions,"Complimenting, expressing appreciation or criticism",Expressing agreement or disagreement,"Phatics, salutations",prediction
0,1604198662-928,Hi did you fix this bug?\n,bugreports,Subject 11057234,NaN,NaN,True,NaN,0,0,0,0,1,0,0,1,0
1,1604198796-1345,I have never been quite certain that the X ax...,bugreports,RFE: An easy way to read off feature frequencies,NaN,NaN,NaN,NaN,0,0,1,0,1,0,0,0,0
2,1604198776-1275,\nBecause I can understand it is of immense in...,bugreports,Statistics not updating on Gravity Spy home page,NaN,NaN,True,😦,1,0,1,0,0,0,0,0,1
3,1604198884-1593,"And working again.\n(A Schrödinger tool, as we...",bugreports,Gravity Spy Tools down,NaN,NaN,NaN,😃,1,1,1,0,0,0,0,0,1
4,1604197902-62,I did occasionally. Reloading the site helps t...,bugreports,Already Seen keeps coming up,/users/eperozzi;/users/sbc538,@eperozzi;@sbc538,NaN,NaN,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,1604745498-12981,I think just a #1080 but a bit of #helix in th...,notes,Subject 5797761,/projects/zooniverse/gravity-spy/talk/subjects...,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0
396,1604529123-3128,#repeatingblips\n,notes,Subject 5864682,/projects/zooniverse/gravity-spy/talk/subjects...,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0
397,1604881637-6577,Another 1080 line with an amplitude envelope.\n,notes,Subject 5956891,/projects/zooniverse/gravity-spy/talk/subjects...,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0
398,1604538743-2305,Very low frequency blob 16-32Hz\n,notes,Subject 25771717,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0


In [ ]:
all_data['prediction'] = model.predict(all_data['text'].apply(clean_tweets))
all_data['prediction'] = (all_data['prediction'].astype(bool) | ~pd.isnull(all_data['emojis'])).astype(int)

In [ ]:
predictions_df = all_data[all_data['prediction'] == 1]
print(len(predictions_df))
np.unique(predictions_df['section'], return_counts=True)

13321


(array(['bugreports', 'chat', 'collections', 'help', 'ligo', 'notes',
        'science', 'virgo'], dtype=object),
 array([  236,   563,   163,   735,    84, 11213,   318,     9]))

In [ ]:
all_data.to_csv('emotions_predicted.csv', index=False)